In [45]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv('tmdb-movies.csv')

column_keys = ['original_title','director','keywords','genres'] # vector dimentions

columns = df.columns.tolist()
#cleanning data
for column in columns:
  df[column]=df[column].fillna('')

def combine(row):
  # creating new column in dataframe containing
  return "#".join([str(row[column]) for column in column_keys])

def dataFrame():
  # creating new column in dataframe containing
  return pd.DataFrame(df, columns=column_keys)

def movie_dataframe(index):
  return pd.DataFrame(df[df.index==index], columns=column_keys)

def get_movie_summary(index):
  return ", ".join(movie_dataframe(index).values[0])

def get_index_from_title(title):
    try:
      return df[df['original_title'] == title].index[0]
    except:
      raise Exception('Movie not found')

df['combined']=df.apply(combine,axis=1)
vector=CountVectorizer()
matrix=vector.fit_transform(df['combined'])    #method to vectorize the text data
print(matrix)
cosine=cosine_similarity(matrix)               #applying the function
print(cosine)

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 133221 stored elements and shape (10866, 16832)>
  Coords	Values
  (0, 7900)	1
  (0, 16466)	1
  (0, 3072)	1
  (0, 15407)	1
  (0, 10054)	1
  (0, 4202)	1
  (0, 15570)	1
  (0, 12534)	1
  (0, 15810)	1
  (0, 7537)	1
  (0, 226)	1
  (0, 280)	1
  (0, 13234)	1
  (0, 5271)	1
  (0, 15071)	1
  (1, 226)	1
  (1, 280)	1
  (1, 13234)	1
  (1, 5271)	1
  (1, 15071)	1
  (1, 9122)	1
  (1, 9513)	1
  (1, 5728)	1
  (1, 12653)	1
  (1, 5909)	1
  :	:
  (10863, 1497)	1
  (10863, 1017)	1
  (10863, 15441)	1
  (10864, 226)	1
  (10864, 3124)	1
  (10864, 16450)	1
  (10864, 441)	1
  (10864, 15706)	1
  (10864, 16254)	1
  (10864, 14177)	1
  (10864, 15122)	1
  (10864, 8799)	1
  (10865, 15002)	1
  (10865, 10779)	1
  (10865, 7057)	1
  (10865, 5335)	1
  (10865, 12950)	1
  (10865, 4405)	1
  (10865, 6399)	1
  (10865, 16108)	1
  (10865, 6585)	1
  (10865, 6521)	1
  (10865, 5156)	1
  (10865, 5384)	1
  (10865, 9296)	1
[[1.         0.31311215 0.26666667 ... 0.         0.08

In [49]:
print("Choose a movie title:")
a = input()

movie_index=get_index_from_title(a)
similar=list(enumerate(cosine[movie_index]))
sorted_movies=sorted(similar, key=lambda x:x[1],reverse=True)  #sorting
#Top ten movies
print(get_movie_summary(sorted_movies[0][0]))
for movie in sorted_movies[1:10]:
  print(movie[1], get_movie_summary(movie[0]))

Choose a movie title:
Alien
Alien, Ridley Scott, android|self-destruction|countdown|space marine|space suit, Horror|Action|Thriller|Science Fiction
0.5477225575051661 Aliens, James Cameron, android|extraterrestrial technology|space marine|spaceman|cryogenics, Horror|Action|Thriller|Science Fiction
0.5407380704358751 Moonraker, Lewis Gilbert, venice|mass murder|space marine|space suit|marcus square, Action|Adventure|Thriller|Science Fiction
0.5303300858899107 Lifeforce, Tobe Hooper, space marine|vampire|flying saucer|comet|alien, Action|Science Fiction|Thriller|Horror|Fantasy
0.5270462766947299 Planet of the Apes, Tim Burton, gorilla|space marine|space suit|revolution|chimp, Thriller|Science Fiction|Action|Adventure
0.5229763603684908 AlienÂ³, David Fincher, prison|android|spacecraft|space marine|imprisonment, Science Fiction|Action|Horror
0.5084751798731268 Babylon 5: A Call to Arms, Michael Vejar, space marine|nightmare|space war|alien life-form|space, Action|Drama|Science Fiction|Adv